In [ ]:
import pandas as pd

factors = pd.read_csv("/Users/vanshaj/Work/GitHub/Quant_Labs/Application/Data/Factor model Data/2025-03_FourFactors_and_Market_Returns_Daily_SurvivorshipBiasAdjusted.csv", parse_dates=['Date'])
factors.head()


,Date,SMB,HML,WML,MF,RF
0,1993-10-01,1.472627,1.259330,-0.421257,NaN,NaN
1,1993-10-04,0.466722,0.433937,-0.695421,-0.930543,0.022014
2,1993-10-05,0.060700,0.785148,-0.823549,-0.296248,0.022014
3,1993-10-06,-0.067928,-1.540675,0.177849,-0.326857,0.022014
4,1993-10-07,-0.291446,0.138117,-0.032755,0.380039,0.022014


In [7]:
assets = pd.read_csv('/Users/vanshaj/Work/GitHub/Quant_Labs/Application/Data/Assets Data/NIFTY 50-26-08-2024-to-26-08-2025.csv')
assets.columns = assets.columns.str.strip()  # Remove any leading/trailing spaces
assets['Date'] = pd.to_datetime(assets['Date'], format='%d-%b-%Y')
assets.head()

,Date,Open,High,Low,Close,Shares Traded,Turnover (₹ Cr)
0,2024-08-26,24906.1,25043.80,24874.70,25010.60,210266456,22166.86
1,2024-08-27,25024.8,25073.10,24973.65,25017.75,223346636,26695.00
2,2024-08-28,25030.8,25129.60,24964.65,25052.35,220419448,26994.19
3,2024-08-29,25035.3,25192.90,24998.50,25151.95,353993900,43221.73
4,2024-08-30,25249.7,25268.35,25199.40,25235.90,638166179,89554.91


In [12]:
merged = pd.merge(assets, factors, on='Date', how='inner')
merged.head()
print(merged)
merged.to_csv('merged_factor_data.csv', index=False)


          Date      Open      High       Low     Close  Shares Traded  \
0   2024-08-26  24906.10  25043.80  24874.70  25010.60      210266456   
1   2024-08-27  25024.80  25073.10  24973.65  25017.75      223346636   
2   2024-08-28  25030.80  25129.60  24964.65  25052.35      220419448   
3   2024-08-29  25035.30  25192.90  24998.50  25151.95      353993900   
4   2024-08-30  25249.70  25268.35  25199.40  25235.90      638166179   
..         ...       ...       ...       ...       ...            ...   
145 2025-03-24  23515.40  23708.75  23433.50  23658.35      311900413   
146 2025-03-25  23751.50  23869.60  23601.40  23668.65      338216746   
147 2025-03-26  23700.95  23736.50  23451.70  23486.85      278590831   
148 2025-03-27  23433.95  23646.45  23412.20  23591.95      510302142   
149 2025-03-28  23600.40  23649.20  23450.20  23519.35      387455531   

     Turnover (₹ Cr)       SMB       HML       WML        MF        RF  
0           22166.86 -0.805112  0.194279 -0.303359

In [19]:
import numpy as np
merged['Simple_Return'] = merged['Close'].pct_change()
merged['Log_Return'] = np.log(merged['Close'] / merged['Close'].shift(1))
merged['Intraday_Return'] = (merged['High'] - merged['Low']) / merged['Low']

merged.head()

,Date,Open,High,Low,Close,Shares Traded,Turnover (₹ Cr),SMB,HML,WML,MF,RF,Simple_Return,Log_Return,Intraday_Return
0,2024-08-26,24906.1,25043.80,24874.70,25010.60,210266456,22166.86,-0.805112,0.194279,-0.303359,0.475529,0.052808,NaN,NaN,0.006798
1,2024-08-27,25024.8,25073.10,24973.65,25017.75,223346636,26695.00,0.770331,0.043097,0.073810,0.011850,0.017600,0.000286,0.000286,0.003982
2,2024-08-28,25030.8,25129.60,24964.65,25052.35,220419448,26994.19,-0.361312,-0.282176,0.082713,0.011446,0.017600,0.001383,0.001382,0.006607
3,2024-08-29,25035.3,25192.90,24998.50,25151.95,353993900,43221.73,-1.207262,0.079925,-0.267680,-0.149429,0.017588,0.003976,0.003968,0.007776
4,2024-08-30,25249.7,25268.35,25199.40,25235.90,638166179,89554.91,0.221964,-0.017888,0.289017,0.524694,0.017588,0.003338,0.003332,0.002736


In [21]:
merged_clean = merged.dropna().reset_index(drop=True)
merged_clean.head()
merged_clean.to_csv('merged_factor_data.csv', index=False)

In [23]:
import statsmodels.api as sm
df = pd.read_csv('merged_factor_data.csv', parse_dates=['Date'], index_col='Date')
df['Excess_Return'] = df['Log_Return'] - df['RF']
X = df[['SMB', 'HML', 'WML', 'MF']]
y = df['Excess_Return']
X = sm.add_constant(X)  # adds intercept (alpha)
model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:          Excess_Return   R-squared:                       0.330
Model:                            OLS   Adj. R-squared:                  0.312
Method:                 Least Squares   F-statistic:                     17.76
Date:                Wed, 03 Sep 2025   Prob (F-statistic):           7.12e-12
Time:                        17:59:44   Log-Likelihood:                 425.94
No. Observations:                 149   AIC:                            -841.9
Df Residuals:                     144   BIC:                            -826.9
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0239      0.001    -20.489      0.0